In [1]:
from djitellopy import Tello
from time import sleep
import numpy as np

In [41]:
tello = Tello()

tello.connect(False)


[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 438 - Send command: 'command'
[INFO] tello.py - 462 - Response command: 'ok'


In order to determine the angle at which the drone has to turn at every single point, treat the movement of the drone as a vector and each turn will be calculated with a rotation matrix
https://en.wikipedia.org/wiki/Rotation_matrix

The formula to find the angle between vector u and vector v in 2D space is:
    cos(theta) = (u dot v)/(mag(u) * mag(v))

In [2]:
%store -r vector_smallest_route

def generate_flight_info(route):
    
    if abs(route[0][0]) > abs(route[0][1]):
        if route[0][0] < 0:
            start_vector = [1,0]
        else: 
            start_vector = [-1,0]
        
    elif abs(route[0][0]) < abs(route[0][1]):
        if route[0][1] < 0:
            start_vector = [0,1]
        else: 
            start_vector = [0,-1]
    
    start_vec = np.array(start_vector)
    
    flight_info = []
    #Find angle between the start_vector and every vector in vector_smallest_route. Start_vector is updated after every calculation
    for vector in route:
        vector = np.array(vector)
        dot_product = np.vdot(start_vector, vector)
        start_vector_mag = np.linalg.norm(start_vector)
        vector_mag = np.linalg.norm(vector)
        arccos_val = np.arccos([dot_product/(start_vector_mag*vector_mag)])
        yaw_angle = arccos_val[0]
        time_of_flight = abs(vector[0]*100000)
        yaw_distance_cm = 1.5*yaw_angle
        yaw_speed = yaw_distance_cm/time_of_flight
        flight_info.append([int(time_of_flight), int(yaw_speed)])
    return [start_vector, flight_info]

In [14]:
%store -r test_route_vector

In [3]:
flight_info = generate_flight_info(vector_smallest_route)
flight_info


[[-1, 0],
 [[52, 0], [13, 0], [13, 0], [52, 0], [53, 0], [12, 0], [12, 0], [53, 0]]]

In [35]:
test_height = "7dm"
height = test_height[:test_height.index('d')]
height = int(height)
height

7

In [43]:
# tello.query_battery()
tello.send_command_with_return("takeoff", 3)

height = tello.send_read_command("height?")
height = height[:height.index('d')]
height = int(height)

if height < 200:
    tello.send_command_with_return("up " + str(200-height), 3)

    
# test_flight = flight_info[1][:2]
# # print(test_flight)
# for route_vector in test_flight:
#     tello.send_rc_control(0, 100, 0, route_vector[1])
#     sleep(route_vector[0])

tello.send_command_with_return("forward 70", 4)
tello.send_command_with_return("cw 180", 4)
tello.send_command_with_return("forward 70", 4)

# tello.send_rc_control(0, 35, 0, 0)
# sleep(3)
# tello.send_rc_control(0, 0, 0, 0)
# sleep(1)
# tello.send_rc_control(0, 0, 0, 45)
# sleep(6)
# tello.send_rc_control(0, 0, 0, 0)
# sleep(1)
# tello.send_rc_control(0, 35, 0, 0)
# sleep(2)
# tello.send_rc_control(0, 0, 0, 0)
# sleep(1)

tello.land()

# starting_lat = 47.5844989
# starting_lon = -122.3009966

[INFO] tello.py - 438 - Send command: 'takeoff'
[INFO] tello.py - 462 - Response takeoff: 'ok'
[INFO] tello.py - 438 - Send command: 'height?'
[INFO] tello.py - 462 - Response height?: '0dm'
[INFO] tello.py - 438 - Send command: 'up 200'
[INFO] tello.py - 462 - Response up 200: 'error Motor stop'
[INFO] tello.py - 438 - Send command: 'forward 70'
[INFO] tello.py - 462 - Response forward 70: 'error No valid imu'
[INFO] tello.py - 438 - Send command: 'cw 180'
[INFO] tello.py - 462 - Response cw 180: 'error No valid imu'
[INFO] tello.py - 438 - Send command: 'forward 70'
[INFO] tello.py - 462 - Response forward 70: 'error No valid imu'
[INFO] tello.py - 438 - Send command: 'land'
[WARNING] tello.py - 448 - Aborting command 'land'. Did not receive a response after 7 seconds
[INFO] tello.py - 438 - Send command: 'land'
[INFO] tello.py - 462 - Response land: 'ok'
